In [ ]:
# cell 1
import sys
import os

# Set up the path to include the src directory
sys.path.append(os.path.abspath('..'))


%load_ext autoreload
%autoreload 2

# Import our custom modules
from src.utils import get_spark_session
from src.config import API_KEY, BASE_URL, MOVIE_IDS
from src.ingestion import fetch_movie_data
from src.cleaning import clean_movie_data
from src.analysis import get_ranked_movies, analyze_franchises
from src.visualization import (
    plot_yearly_trends, 
    plot_genre_roi, 
    plot_revenue_vs_budget, 
    plot_franchise_comparison, 
    plot_popularity_vs_rating
)

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Initialize Spark
spark = get_spark_session("TMDB_Analysis_Lab")
print(f"Spark Version: {spark.version}")

SUCCESS: API Key loaded.
Spark Version: 3.5.0


In [2]:
# cell 2
# 1. Fetch raw data using Python (Driver Node)
print(f"Fetching data for {len(MOVIE_IDS)} movies...")
raw_data_list = fetch_movie_data(MOVIE_IDS, API_KEY, BASE_URL)

# 2. Convert to Spark DataFrame
# Spark automatically infers the schema from the list of dicts
df_raw = spark.createDataFrame(raw_data_list)

print("Raw Schema:")
df_raw.printSchema()

Fetching data for 19 movies...
Fetching 1/19: ID 0...
Fetching 2/19: ID 299534...
Fetching 3/19: ID 19995...
Fetching 4/19: ID 140607...
Fetching 5/19: ID 299536...
Fetching 6/19: ID 597...
Fetching 7/19: ID 135397...
Fetching 8/19: ID 420818...
Fetching 9/19: ID 24428...
Fetching 10/19: ID 168259...
Fetching 11/19: ID 99861...
Fetching 12/19: ID 284054...
Fetching 13/19: ID 12445...
Fetching 14/19: ID 181808...
Fetching 15/19: ID 330457...
Fetching 16/19: ID 351286...
Fetching 17/19: ID 109445...
Fetching 18/19: ID 321612...
Fetching 19/19: ID 260513...
Raw Schema:
root
 |-- adult: boolean (nullable = true)
 |-- backdrop_path: string (nullable = true)
 |-- belongs_to_collection: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- budget: long (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: long (valueContainsNull = true)
 |-- homepage: stri

In [3]:
# cell 3
# Apply our Spark cleaning pipeline
df_clean = clean_movie_data(df_raw)

# Cache the result since we will use this dataframe for multiple analyses
df_clean.cache()

print(f"Cleaned Row Count: {df_clean.count()}")
df_clean.select("title", "release_date", "revenue_musd", "roi").show(5, truncate=False)

Cleaned Row Count: 18
+----------------------------+------------+------------+-----------------+
|title                       |release_date|revenue_musd|roi              |
+----------------------------+------------+------------+-----------------+
|Avengers: Endgame           |2019-04-24  |2799.4391   |7.86359297752809 |
|Avatar                      |2009-12-16  |2923.706026 |12.33631234599156|
|Star Wars: The Force Awakens|2015-12-15  |2068.223624 |8.441729077551022|
|Avengers: Infinity War      |2018-04-25  |2052.415039 |6.841383463333333|
|Titanic                     |1997-11-18  |2264.162353 |11.320811765     |
+----------------------------+------------+------------+-----------------+
only showing top 5 rows



In [4]:
# cell 4
print("--- Top 5 Highest Revenue ---")
top_rev = get_ranked_movies(df_clean, "revenue_musd", ascending=False)
top_rev.select("title", "revenue_musd", "budget_musd").show()

print("--- Top 5 Highest ROI ---")
top_roi = get_ranked_movies(df_clean, "roi", ascending=False)
top_roi.select("title", "roi", "revenue_musd").show()

print("--- Worst 5 Flops (Lowest ROI) ---")
# Filter for significant budget first to avoid divide-by-zero anomalies on micro-films
flop_roi = get_ranked_movies(df_clean.filter("budget_musd > 10"), "roi", ascending=True)
flop_roi.select("title", "roi", "budget_musd", "revenue_musd").show()

--- Top 5 Highest Revenue ---
+--------------------+------------+-----------+
|               title|revenue_musd|budget_musd|
+--------------------+------------+-----------+
|              Avatar| 2923.706026|      237.0|
|   Avengers: Endgame|   2799.4391|      356.0|
|             Titanic| 2264.162353|      200.0|
|Star Wars: The Fo...| 2068.223624|      245.0|
|Avengers: Infinit...| 2052.415039|      300.0|
+--------------------+------------+-----------+

--- Top 5 Highest ROI ---
+--------------------+-----------------+------------+
|               title|              roi|revenue_musd|
+--------------------+-----------------+------------+
|              Avatar|12.33631234599156| 2923.706026|
|             Titanic|     11.320811765| 2264.162353|
|      Jurassic World|      11.14358296| 1671.537444|
|Harry Potter and ...|     10.732089752| 1341.511219|
|           Frozen II|9.691223173333332| 1453.683476|
+--------------------+-----------------+------------+

--- Worst 5 Flops (Lowes

In [5]:
# cell 5
# Compare Franchises vs Standalone
franchise_stats = analyze_franchises(df_clean)

# Collect to Pandas for display
pdf_franchise = franchise_stats.toPandas()
display(pdf_franchise)

,is_franchise,count,avg_revenue,median_roi,avg_budget,avg_popularity,avg_rating
0,False,18,1691.831828,7.888409,213.777778,25.066233,7.396333


In [6]:
# Cell 6: Data Preparation for Visualizations
from pyspark.sql.functions import year, explode, split, col, median, mean, count

# 1. Prepare Yearly Trends Data
# Extract year from release_date and aggregate
df_yearly = df_clean.withColumn("year", year("release_date")) \
    .groupBy("year") \
    .agg(
        count("id").alias("movie_count"),
        mean("revenue_musd").alias("mean_revenue"),
        mean("budget_musd").alias("mean_budget"),
        mean("roi").alias("mean_roi")
    ).orderBy("year")

# 2. Prepare Genre Data (Explode the 'A|B|C' string into separate rows)
df_genre = df_clean.withColumn("genre", explode(split("genres", "\|"))) \
    .groupBy("genre") \
    .agg(median("roi").alias("median_roi")) \
    .orderBy(col("median_roi").desc())

# 3. Prepare Scatter Plot Data (Individual Movies)
# We select only what we need for the scatter plots to keep it light
df_scatter = df_clean.select("title", "budget_musd", "revenue_musd", "popularity", "vote_average")

# 4. Prepare Franchise Data
# (Re-using the logic from your analyze_franchises function, but explicit here for plotting)
df_franchise = df_clean.withColumn("type", 
                                   F.when(col("belongs_to_collection").isNotNull(), "Franchise")
                                   .otherwise("Standalone")) \
    .groupBy("type") \
    .agg(
        mean("revenue_musd").alias("mean_revenue"),
        mean("budget_musd").alias("mean_budget"),
        mean("roi").alias("mean_roi"),
        mean("vote_average").alias("mean_rating")
    )

# --- Collect all to Pandas ---
pdf_yearly = df_yearly.toPandas()
pdf_genre = df_genre.toPandas()
pdf_scatter = df_scatter.toPandas()
pdf_franchise = df_franchise.toPandas()

print("Data ready for visualization!")

NameError: name 'F' is not defined

In [10]:
# Cell 7
plot_yearly_trends(pdf_yearly)

NameError: name 'plot_yearly_trends' is not defined

In [ ]:
# Cell 8
plot_genre_roi(pdf_genre)

NameError: name 'pdf_genre' is not defined

<Figure size 1400x700 with 0 Axes>

In [9]:
# Cell 9: Revenue vs. Budget Trends
plt.figure(figsize=(10, 6))

plt.scatter(pdf_scatter['budget_musd'], pdf_scatter['revenue_musd'], 
            alpha=0.6, s=100, color='skyblue', edgecolors='grey', label='Individual Movies')

plt.title('Revenue vs. Budget Trends', fontsize=16)
plt.xlabel('Budget (Million USD)', fontsize=12)
plt.ylabel('Revenue (Million USD)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()

plt.show()

NameError: name 'pdf_scatter' is not defined

<Figure size 1000x600 with 0 Axes>

In [ ]:
# Cell 10: Franchise vs Standalone Performance
fig, axes = plt.subplots(2, 2, figsize=(18, 10))
fig.suptitle('Franchise vs Standalone Movie Performance', fontsize=14, weight='bold')

# Helper function to replicate the specific bar style
def plot_compare(ax, y_col, title, ylabel):
    sns.barplot(data=pdf_franchise, x='type', y=y_col, ax=ax, palette=['#2b8cbe', '#a53e74'], edgecolor='black')
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_xlabel('')
    ax.grid(axis='y', linestyle='-', alpha=0.3)

# 1. Revenue
plot_compare(axes[0, 0], 'mean_revenue', 'Average Revenue (M USD)', 'Million USD')

# 2. ROI
plot_compare(axes[0, 1], 'mean_roi', 'Average ROI', 'ROI Multiplier')

# 3. Budget
plot_compare(axes[1, 0], 'mean_budget', 'Average Budget (M USD)', 'Million USD')

# 4. Rating
plot_compare(axes[1, 1], 'mean_rating', 'Average Rating', 'Rating (out of 10)')
axes[1, 1].set_ylim(0, 10) # Fix y-axis to 10 for ratings

plt.show()

In [ ]:
# Cell 11: Relationship: Popularity vs. User Rating
plt.figure(figsize=(10, 6))

plt.scatter(pdf_scatter['popularity'], pdf_scatter['vote_average'], 
            alpha=0.7, s=100, color='#a05195', edgecolors='black', label='Individual Movies')

plt.title('Relationship: Popularity vs. User Rating', fontsize=16)
plt.xlabel('Popularity Score', fontsize=12)
plt.ylabel('Vote Average (0-10)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()

plt.show()